In [1]:
# delete all rules created N months ago
from datetime import datetime, timedelta
from rucio.client import Client


In [2]:
days=0

In [3]:
rucio=Client()
me=rucio.whoami()
account=me['account']
print(f"using Rucio account: {account}")

using Rucio account: belforte


In [4]:
# get rules for this account
ruleGen = rucio.list_replication_rules({'account': account})
rules = list(ruleGen)
print(f"{len(rules)} rules exist for account: {account}")

10 rules exist for account: belforte


In [5]:
def getRuleQuota(id):
    """ return quota needed by this rule in MB """
    size = 0
    ruleG = rucio.list_replication_rules({'id':id})
    try:
        rule=next(ruleG)
        replicaG = rucio.list_replicas([{'scope': rule['scope'], 'name': rule['name']}])
        for rep in replicaG:
            size += rep['bytes']
    except StopIteration:
        pass

    return size/1e6

In [6]:
id='a256abc2f93f40eba5b3b0107d686956'
quota = getRuleQuota(id)
print(quota)

0.0


In [8]:
now = datetime.now()
totalSize = 0
totalRules = 0
for rule in rules:
    created=rule['created_at']
    size = getRuleQuota(rule['id'])
    isOld = (now - created) > timedelta(days=days)
    if isOld:
        totalRules +=1
        totalSize += size
        print(f"will delete rule {rule['id']} created on {created} - {size:10.3f} MB")
        #rr=rucio.delete_replication_rule(rule_id=rule['id'], purge_replicas=True)
        #if rr:
        #    print(f"deleted rule {rule['id']} created on {created}")
        #else:
        #    print(f"ERROR deletion of rule {rule['ruleId']} failed")
              
print(f"TOTAL: {totalRules} rules - {totalSize:.0f} MB = {totalSize/1e3:0.3f} GB")

will delete rule f9302bd5e3564c878d2cefd61322ad3a created on 2024-01-24 14:55:42 -      0.000 MB
will delete rule d76572004201445aa4405d22104a12df created on 2024-01-24 21:16:32 -   7733.942 MB
will delete rule 6e1b274e03ae4278bc8f94ec67eba3d8 created on 2024-01-24 21:41:25 -   7733.942 MB
will delete rule c14a00c2c76b491d9588cc7152145583 created on 2024-02-01 21:59:09 -   2845.587 MB
will delete rule 579e577534654beeadd5c661b98714c3 created on 2024-02-01 22:03:44 -   4103.823 MB
will delete rule 84036d7f4a8f42cab7ac554d7dfdd4d2 created on 2024-02-01 21:49:05 -   2542.482 MB
will delete rule ab5542538c93491dbe9b91fce7742961 created on 2024-02-01 21:43:08 -     27.874 MB
will delete rule fe3399d5fce446049d05130ffd79780a created on 2024-02-01 21:43:08 -     27.874 MB
will delete rule 79a0d812e7cf4cac8ac8a7d46bbc5f0d created on 2024-02-01 21:59:09 -   2845.587 MB
will delete rule 1f6fe272bce74c7680c637b93133bdf5 created on 2024-02-01 22:03:44 -   4103.823 MB
TOTAL: 10 rules - 31965 MB = 3